In [2]:
import numpy as np
import tensorflow as tf

import deepchem as dc
from deepchem.models import GraphConvModel
from deepchem.molnet import load_delaney

In [3]:
delaney_tasks = [ 'measured log solubility in mols per litre' ]
featurizer = dc.feat.ConvMolFeaturizer()
loader = dc.data.CSVLoader( tasks=delaney_tasks, feature_field="smiles", featurizer=featurizer )
dataset_file = '../data/delaney-processed.csv'
dataset = loader.featurize( dataset_file, shard_size=8192 )

C:\ProgramData\Anaconda3\envs\rdkit\lib\site-packages\deepchem\data\data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)


In [4]:
transformers = [ dc.trans.NormalizationTransformer( transform_y = True, dataset=dataset ) ]
for transformer in transformers:
    dataset = transformer.transform(dataset)

splitter = dc.splits.IndexSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

In [5]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)

batch_size = 128
model = GraphConvModel( len(delaney_tasks), batch_size=batch_size, mode='regression')

In [6]:
model.fit(train_dataset, nb_epoch=100)

C:\ProgramData\Anaconda3\envs\rdkit\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.021954967975616455

In [7]:
print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)

print("Train scores :", train_scores)
print("Validation scores : ", valid_scores)

Evaluating model


n_samples is a deprecated argument which is ignored.
n_samples is a deprecated argument which is ignored.


Train scores : {'mean-pearson_r2_score': 0.98989021882828}
Validation scores :  {'mean-pearson_r2_score': 0.7573890156036788}
